In [2]:
# import pandas as pd
# import nltk
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords

# from nltk.tokenize import RegexpTokenizer
# import re
# from gensim.models import Word2Vec
# from gensim.models.phrases import Phrases, Phraser
# from gensim.models import KeyedVectors
# from PIL import Image
# from io import BytesIO
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
book_data=pd.read_csv('book_data.csv', index_col=0)

In [5]:
book_data["Desc"]

0       We know that power is shifting: From West to E...
1       Following the success of The Accidental Billio...
2       How to tap the power of social software and ne...
3       William J. Bernstein is an American financial ...
4       Amazing book. And I joined Steve Jobs and many...
                              ...                        
2446    Ralph Roberts, a sus setenta años y tras la mu...
2447    Murder at the Vicarage marks the debut of Agat...
2448    In 1951 John Wyndham published his novel The D...
2449    This now classic book revealed Flannery O'Conn...
2450    Imbued on every page with Frank McCourt's asto...
Name: Desc, Length: 2382, dtype: object

In [8]:
book_data.column_names

AttributeError: 'DataFrame' object has no attribute 'column_names'

In [5]:
book_data.head()

,Desc,Unnamed: 0.1,author,genre,image_link,rating,title
0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [10]:
book_data.columns

Index(['Desc', 'Unnamed: 0.1', 'author', 'genre', 'image_link', 'rating',
       'title'],
      dtype='object')

In [17]:
book_data=book_data.drop(columns=['Unnamed: 0.1'])

In [18]:
book_data.genre.unique()

array(['Business', 'Non-Fiction'], dtype=object)

In [22]:
#Utitlity functions for removing ASCII characters, converting lower case, removing stop words, html and punctuation from description

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

book_data['cleaned'] = book_data['Desc'].apply(_removeNonAscii)

book_data['cleaned'] = book_data.cleaned.apply(func = make_lower_case)
book_data['cleaned'] = book_data.cleaned.apply(func = remove_stop_words)
book_data['cleaned'] = book_data.cleaned.apply(func=remove_punctuation)
book_data['cleaned'] = book_data.cleaned.apply(func=remove_html)


In [24]:
#splitting the description into words

corpus = []
for descriptions in book_data['cleaned']:
    corpus.append(descriptions.split())

In [40]:
# Downloading the Google pretrained Word2Vec Model
# ! wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
# ! wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Training our corpus with Google Pretrained Model

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

(0, 0)

In [41]:
# Generate the average word2vec for the each book description

def vectors(x):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description 
    for line in book_data['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)

In [49]:
# Recommending the Top 5 similar books

def recommendations(title):
    
    # Calling the function vectors

    array_embeddings= vectors(book_data)
    
    # finding cosine similarity for the vectors

    cosine_similarities = cosine_similarity(array_embeddings, array_embeddings)

    # taking the title and book image link and store in new data frame called books
    books = book_data[['title', 'image_link']]
    #Reverse mapping of the index
    indices = pd.Series(book_data.index, index = book_data['title']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]
    for index, row in recommend.iterrows():

        response = requests.get(row['image_link'])
        img = Image.open(BytesIO(response.content))
        plt.figure()
        plt.imshow(img)
        plt.title(row['title'])

In [50]:
recommendations("The Da Vinci Code")


/Users/claireh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/claireh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.